合并结果：
1. 将interpret-cxr的train dev test，以及findings impression整理为raw_reports.json
	1. /home/yuxiang/liao/workspace/arrg_data_processing/1_get_raw_json.ipynb  (Interpret-cxr docs) 
	2. 输入为interpret-cxr
	3. 输出为：outputs/interpret_report/raw.json 
2. 利用llm，将句子进行拆分，移除连词
	1. /home/yuxiang/liao/workspace/arrg_sentgen/llm_split_sent.py 
	2. 输入为step1
	3. 输出 outputs/interpret_cxr/llm_sent_splits_1_of_3.json 
3. 将拆分后的句子进行bioportal
	1. /home/yuxiang/liao/workspace/arrg_report2graph/1_bioportal_annotate_4sents.py
	2. 输入为step2，
	3. 输出为 outputs/interpret_sents/bioportal_annotated_radlex/train_dev_test.jsonlines 
4. 对拆分后的句子重新整理，作为cxrgraph和radcoref的输入
	1. /home/yuxiang/liao/workspace/arrg_data_processing/1_get_raw_json.ipynb  (Interpret-cxr sentences) 
	2. 输入为step2，
	3. 输出为 outputs/interpret_sents/raw4cxrgraph.json
5. cxrgraph
	1. /home/yuxiang/liao/workspace/arrg_cxrgraph/inference/inference.sh
	2. 输入step4
	3. 输出：outputs/interpret-cxr-sents/inference.json
6. radcoref
	1. /home/yuxiang/liao/workspace/fast-coref/src/inference/custom_inference.py
	2. 输入 step4
	3. 输出：outputs/interpret_sents/coref_inference.json

In [25]:
raw_reports = "/home/yuxiang/liao/workspace/arrg_data_processing/outputs/interpret_report/raw.json"
llm_sents = "/home/yuxiang/liao/workspace/arrg_sentgen/outputs/interpret_cxr/llm_sent_splits_1_of_3.json"
biop_sents = "/home/yuxiang/liao/workspace/arrg_report2graph/outputs/interpret_sents/bioportal_annotated_radlex/train_dev_test.jsonlines"
raw_sents = "/home/yuxiang/liao/workspace/arrg_data_processing/outputs/interpret_sents/raw4cxrgraph.json"
cxrg_sents = "/home/yuxiang/liao/workspace/arrg_cxrgraph/outputs/interpret-cxr-sents/inference.json"
radcoref_sents = "/home/yuxiang/liao/workspace/fast-coref/outputs/interpret_sents/coref_inference.json"

In [61]:
import datasets
from datasets import load_dataset, Sequence, Image, DatasetDict, concatenate_datasets, Dataset
import os
import json
from tqdm import tqdm
import re
import copy

In [22]:
temp_dir = "/home/yuxiang/liao/workspace/arrg_preprocessing/outputs/interpret_sents/combined_results"


def save_temp_ds(ds, version=1):
    if version == 1:
        temp_path = os.path.join(temp_dir, "temp_v1")
    else:
        temp_path = os.path.join(temp_dir, "temp_v2")
    ds.save_to_disk(temp_path)
    return temp_path

## Load the original dataset: interpret-cxr

In [4]:
data_dir = "/home/yuxiang/liao/mimic/mimic-cxr-jpg/2.1.0"
PATH_PREFIX = "/home/yuxiang/liao/mimic/mimic-cxr-jpg/2.1.0"

dataset_interpret = datasets.load_from_disk("/home/yuxiang/liao/resources/datasets/interpret-cxr")
dataset_interpret_test_public = datasets.load_from_disk("/home/yuxiang/liao/resources/datasets/interpret-cxr-test-public")
dataset_mimic = load_dataset("json", data_files={"train": os.path.join(data_dir, "train_mimic.json"), "validation": os.path.join(data_dir, "val_mimic.json")})


def add_prefix(example):
    example["images"] = [os.path.join(PATH_PREFIX, i) for i in example["images"]]
    return example


dataset_mimic = dataset_mimic.map(add_prefix, num_proc=8).cast_column("images", Sequence(Image()))
dataset_final = DatasetDict({"train": concatenate_datasets([dataset_interpret["train"], dataset_mimic["train"]]), "validation": concatenate_datasets([dataset_interpret["validation"], dataset_mimic["validation"]]), "test": dataset_interpret_test_public["test"]})
# We don't need this column at the moment. It will slowdown the iter speed
dataset_final = dataset_final.remove_columns(["images"])

In [5]:
dataset_final

DatasetDict({
    train: Dataset({
        features: ['source', 'images_path', 'impression', 'findings'],
        num_rows: 550395
    })
    validation: Dataset({
        features: ['source', 'images_path', 'impression', 'findings'],
        num_rows: 14111
    })
    test: Dataset({
        features: ['findings', 'impression'],
        num_rows: 3677
    })
})

Convert dataset row scale from "study" to "section"

In [6]:
def dataset_generator():
    for split in ["train", "validation", "test"]:
        for idx, data in enumerate(dataset_final[split]):
            doc_key_prefix = f"{split}#{idx}"
            yield {"doc_key": f"{doc_key_prefix}#findings", "original_text": data["findings"]}
            yield {"doc_key": f"{doc_key_prefix}#impression", "original_text": data["impression"]}

In [7]:
new_ds = Dataset.from_generator(dataset_generator)

Generating train split: 1136366 examples [00:17, 63676.15 examples/s]


In [11]:
temp_path = save_temp_ds(new_ds, version=1)

Saving the dataset (1/1 shards): 100%|██████████| 1136366/1136366 [00:00<00:00, 4769741.60 examples/s]


## 将spacy分词后的结果加入到新数据集中

In [12]:
with open(raw_reports, "r") as f:
    docs = [json.loads(line) for line in f]

In [13]:
new_column = []
for idx, doc in enumerate(tqdm(docs)):
    if doc["sentences"] != []:
        first_tok = doc["sentences"][0][0]
        assert first_tok == new_ds[idx]["original_text"][: len(first_tok)], f"doc: {doc}, ds: {new_ds[idx]}"
    new_column.append(doc["sentences"])

100%|██████████| 1136366/1136366 [00:19<00:00, 57313.99it/s]


In [14]:
new_ds = new_ds.add_column("original_sentences", new_column)
new_ds

In [15]:
temp_path = save_temp_ds(new_ds, version=2)

Saving the dataset (1/1 shards): 100%|██████████| 1136366/1136366 [00:01<00:00, 799517.24 examples/s] 


Dataset({
    features: ['doc_key', 'original_text', 'original_sentences'],
    num_rows: 1136366
})

## 将llm拆分后的原始句子加入新数据集

注意：
1. 前后的空白字符
2. 空句子
3. 

In [16]:
new_ds = datasets.load_from_disk(temp_path)
new_ds

Dataset({
    features: ['doc_key', 'original_text', 'original_sentences'],
    num_rows: 1136366
})

In [17]:
docKey2splitSentDict = {doc_key: {} for doc_key in new_ds["doc_key"]}


def read_file_generator(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            out = line.strip()
            yield json.loads(out)


for partition in [1, 2, 3]:
    input_file_path = os.path.join(os.path.dirname(llm_sents), f"llm_sent_splits_{partition}_of_3.json")
    for doc in tqdm(read_file_generator(input_file_path)):
        # doc["sent_idx"] could be None, if the original text is None
        docKey2splitSentDict[doc["doc_key"]][doc["sent_idx"]] = doc["sent_splits"]

991884it [00:02, 471560.84it/s]
1051578it [00:03, 280290.14it/s]
1231038it [00:04, 288930.04it/s]


In [18]:
new_column = []

for row_idx, (doc_key, sent_dict) in enumerate(tqdm(docKey2splitSentDict.items())):
    if None in sent_dict:
        new_column.append([])
        continue
    else:
        split_text = []
        for sent_idx, split_sent_list in sorted(sent_dict.items(), key=lambda x: x[0]):
            # for split_sent in split_sent_list:
            #     assert len(split_sent) == len(split_sent.strip()), f"{split_sent} || {sent_dict}"
            split_text.append(split_sent_list)

        assert new_ds[row_idx]["doc_key"] == doc_key
        assert len(split_text) == len(new_ds[row_idx]["original_sentences"])
        new_column.append(split_text)

100%|██████████| 1136366/1136366 [01:24<00:00, 13404.21it/s]


In [20]:
new_ds = new_ds.add_column("split_text", new_column)
new_ds

Dataset({
    features: ['doc_key', 'original_text', 'original_sentences', 'split_text'],
    num_rows: 1136366
})

In [23]:
temp_path = save_temp_ds(new_ds, version=1)

Saving the dataset (2/2 shards): 100%|██████████| 1136366/1136366 [00:00<00:00, 2076117.98 examples/s]


## 解析Bioportal的结果

In [26]:
with open(biop_sents, "r") as f:
    docs = [json.loads(line) for line in f]

In [30]:
doc = docs[0]
doc

{'ids': ['train#0#impression#0@0',
  'train#0#impression#0@1',
  'train#0#impression#1@0',
  'train#0#impression#2@0',
  'train#0#impression#2@1',
  'train#1#findings#0@0',
  'train#2#findings#0@0',
  'train#2#findings#0@1',
  'train#2#findings#0@2',
  'train#3#findings#0@0',
  'train#3#findings#0@1',
  'train#3#findings#1@0',
  'train#3#findings#1@1',
  'train#3#findings#1@2',
  'train#3#findings#2@0',
  'train#3#findings#3@0',
  'train#3#findings#3@1'],
 'sents': ['Decreased bibasilar parenchymal opacities are seen.',
  'The bibasilar parenchymal opacities are now minimal.',
  'Stable small left pleural effusion.',
  'Feeding tube is again seen.',
  'Sternal plates are again seen.',
  "No pathological alterations are identified for the patient's age.",
  'Normal cardiomedastinal silhouette.',
  'No evidence of pulmonary infiltrates.',
  'No occupation of the costophrenic sinuses.',
  'A chest radiograph on 12/11/08 at 2315 is demonstrated.',
  'A dual lead AICD is demonstrated.',
  '

In [43]:
ann_result = json.loads(doc["ann"])
ann_result

[{'annotatedClass': {'@id': 'http://radlex.org/RID/RID36044',
   '@type': 'http://www.w3.org/2002/07/owl#Class'},
  'hierarchy': [],
  'annotations': [{'from': 7,
    'to': 15,
    'matchType': 'PREF',
    'text': 'DECREASED'}],
  'mappings': []},
 {'annotatedClass': {'@id': 'http://radlex.org/RID/RID5670',
   '@type': 'http://www.w3.org/2002/07/owl#Class'},
  'hierarchy': [],
  'annotations': [{'from': 116,
    'to': 122,
    'matchType': 'PREF',
    'text': 'MINIMAL'}],
  'mappings': []},
 {'annotatedClass': {'@id': 'http://radlex.org/RID/RID5734',
   '@type': 'http://www.w3.org/2002/07/owl#Class'},
  'hierarchy': [],
  'annotations': [{'from': 138,
    'to': 143,
    'matchType': 'PREF',
    'text': 'STABLE'},
   {'from': 625, 'to': 630, 'matchType': 'PREF', 'text': 'STABLE'},
   {'from': 668, 'to': 673, 'matchType': 'PREF', 'text': 'STABLE'}],
  'mappings': []},
 {'annotatedClass': {'@id': 'http://radlex.org/RID/RID5774',
   '@type': 'http://www.w3.org/2002/07/owl#Class'},
  'hiera

In [58]:
new_ds[10]

{'doc_key': 'train#5#findings',
 'original_text': 'Anteroposterior chest. Rotated study. Bibasal hypoventilation.',
 'original_sentences': [['Anteroposterior', 'chest', '.'],
  ['Rotated', 'study', '.'],
  ['Bibasal', 'hypoventilation', '.']],
 'split_text': [['Anteroposterior chest.'],
  ['Rotated study.'],
  ['Bibasal hypoventilation.']]}

In [63]:
# Check if the split text is the same as the original text
pattern = re.compile(r"<\|\d+\|> (.*?) <\|END\|>")
sents_check = pattern.findall(doc["sent_text"])
for sent1, sent2 in zip(sents_check, doc["sents"]):
    assert sent1 == sent2, f"{sent1} || {sent2}"

# Split batched sents text into individual sents
char2sent_idx = [-1] * len(doc["sent_text"])
split_sents_info = []
for idx, (sent_id, split_sent) in enumerate(zip(doc["ids"], doc["sents"])):
    sent_char_start = doc["sent_text"].index(split_sent)
    sent_char_end = sent_char_start + len(split_sent)
    assert doc["sent_text"][sent_char_start:sent_char_end] == split_sent

    char2sent_idx[sent_char_start:sent_char_end] = [idx] * len(split_sent)
    split_sents_info.append(
        {
            "sent_char_start": sent_char_start,  # also the char offset
            "sent_char_end": sent_char_end,
            "split_sent": split_sent,
            "sent_id": sent_id,
            "radlex": [],
        }
    )

"""
ann_result = {
    'annotatedClass': {'@id': 'http://radlex.org/RID/RID5734', '@type': 'http://www.w3.org/2002/07/owl#Class'},
    'hierarchy': [],
    'annotations': [
        {'from': 138, 'to': 143, 'matchType': 'PREF', 'text': 'STABLE'},
        {'from': 625, 'to': 630, 'matchType': 'PREF', 'text': 'STABLE'},
        {'from': 668, 'to': 673, 'matchType': 'PREF', 'text': 'STABLE'}],
    'mappings': []},
"""
for ann_result in json.loads(doc["ann"]):
    full_id = ann_result["annotatedClass"]["@id"]
    for annotation in ann_result["annotations"]:
        from_char = annotation["from"] - 1
        to_char = annotation["to"]
        ann_text = annotation["text"].lower()
        ann_matchType = annotation["matchType"]

        sent_idx = char2sent_idx[from_char]
        assert sent_idx == char2sent_idx[to_char - 1]
        assert ann_text == doc["sent_text"][from_char:to_char].lower()

        split_sent_dict = split_sents_info[sent_idx]
        split_sent_dict["radlex"].append((from_char, to_char, ann_text, ann_matchType, full_id))

In [64]:
split_sents_info

[{'sent_char_start': 6,
  'sent_char_end': 57,
  'split_sent': 'Decreased bibasilar parenchymal opacities are seen.',
  'sent_id': 'train#0#impression#0@0',
  'radlex': [(6, 15, 'decreased', 'PREF', 'http://radlex.org/RID/RID36044')]},
 {'sent_char_start': 71,
  'sent_char_end': 123,
  'split_sent': 'The bibasilar parenchymal opacities are now minimal.',
  'sent_id': 'train#0#impression#0@1',
  'radlex': [(115, 122, 'minimal', 'PREF', 'http://radlex.org/RID/RID5670')]},
 {'sent_char_start': 137,
  'sent_char_end': 172,
  'split_sent': 'Stable small left pleural effusion.',
  'sent_id': 'train#0#impression#1@0',
  'radlex': [(137, 143, 'stable', 'PREF', 'http://radlex.org/RID/RID5734'),
   (144, 149, 'small', 'PREF', 'http://radlex.org/RID/RID5774'),
   (150, 154, 'left', 'PREF', 'http://radlex.org/RID/RID5824'),
   (155, 171, 'pleural effusion', 'PREF', 'http://radlex.org/RID/RID34539'),
   (163, 171, 'effusion', 'PREF', 'http://radlex.org/RID/RID4872')]},
 {'sent_char_start': 186,
  '